# Question Webpage

[PostgreSQL Exercises](https://pgexercises.com/)


In [1]:
import psycopg
import pandas as pd

DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "exercises"
DB_USER = "world"
DB_PASS = "world123"

def execute(query):

    with  psycopg.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST, port=DB_PORT) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    return data

In [3]:
query = """SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';"""
execute(query)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,cd,facilities,world,None,True,False,True,False
1,cd,bookings,world,None,True,False,True,False
2,cd,members,world,None,True,False,True,False


# Basic

## Retrieve everything from a table

How can you retrieve all the information from the cd.facilities table?
[Link](https://pgexercises.com/questions/basic/selectall.html)


In [4]:
query = "select * from cd.facilities limit 10"
execute(query)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5,25,10000,200
1,1,Tennis Court 2,5,25,8000,200
2,2,Badminton Court,0,15.5,4000,50
3,3,Table Tennis,0,5,320,10
4,4,Massage Room 1,35,80,4000,3000
5,5,Massage Room 2,35,80,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0,5,450,15
8,8,Pool Table,0,5,400,15


## Retrieve specific columns from a table

You want to print out a list of all of the facilities and their cost to members. 
How would you retrieve a list of only facility names and costs?
[Link](https://pgexercises.com/questions/basic/selectspecific.html)


In [5]:
query = """
    select name, membercost from cd.facilities limit 10
"""
execute(query)

,name,membercost
0,Tennis Court 1,5
1,Tennis Court 2,5
2,Badminton Court,0
3,Table Tennis,0
4,Massage Room 1,35
5,Massage Room 2,35
6,Squash Court,3.5
7,Snooker Table,0
8,Pool Table,0


## Control which rows are retrieved

You want to only retrive costs that exceed 0
[Link](https://pgexercises.com/questions/basic/where.html)


In [7]:
query = """
    select name, membercost from cd.facilities where membercost > 0 limit 10
"""
execute(query)

,name,membercost
0,Tennis Court 1,5
1,Tennis Court 2,5
2,Massage Room 1,35
3,Massage Room 2,35
4,Squash Court,3.5


## Control which rows are retrieved - part 2

How can you produce a list of facilities that charge a fee to members,
and that fee is less than 1/50th of the monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.
[Link](https://pgexercises.com/questions/basic/where2.html)

In [13]:
query = """
    select facid, name, membercost, monthlymaintenance
    from cd.facilities
    where membercost < monthlymaintenance / 50 and membercost > 0
"""
execute(query)

,facid,name,membercost,monthlymaintenance
0,4,Massage Room 1,35,3000
1,5,Massage Room 2,35,3000



## Basic string searches

How can you produce a list of all facilities with the word 'Tennis' in their name?
[Link](https://pgexercises.com/questions/basic/where3.html)


In [16]:
query = """
    select * from cd.facilities
    where name like '%Tennis%'
"""
execute(query)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5,25,10000,200
1,1,Tennis Court 2,5,25,8000,200
2,3,Table Tennis,0,5,320,10


## Matching against multiple possible values

How can you retrieve the details of facilities with ID 1 and 5?
Try to do it without using the `OR` operator.
[Question](https://pgexercises.com/questions/basic/where4.html)


In [17]:
query = """
    select * from cd.facilities
    where facid in (1,5)
"""
execute(query)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5,25,8000,200
1,5,Massage Room 2,35,80,4000,3000


## Classify results into buckets

How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive'
depending on if their monthly maintenance cost is more than $100?
Return the name and monthly maintenance of the facilities in question.
[Link](https://pgexercises.com/questions/basic/classify.html)


In [19]:
query = """
    select name,
    (
        CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END
    ) as cost
    from cd.facilities
"""
execute(query)

,name,cost
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


## Working with dates

How can you produce a list of members who joined after the start of September 2012? Return the memid,
surname, firstname, and joindate of the members in question.
[Link](https://pgexercises.com/questions/basic/date.html)

In [22]:
query = """
    select memid, surname, firstname, joindate from cd.members where joindate >= '2012-09-01'
"""
execute(query)

,memid,surname,firstname,joindate
0,24,Sarwin,Ramnaresh,2012-09-01 08:44:42
1,26,Jones,Douglas,2012-09-02 18:43:05
2,27,Rumney,Henrietta,2012-09-05 08:42:35
3,28,Farrell,David,2012-09-15 08:22:05
4,29,Worthington-Smyth,Henry,2012-09-17 12:27:15
5,30,Purview,Millicent,2012-09-18 19:04:01
6,33,Tupperware,Hyacinth,2012-09-18 19:32:05
7,35,Hunt,John,2012-09-19 11:32:45
8,36,Crumpet,Erica,2012-09-22 08:36:38
9,37,Smith,Darren,2012-09-26 18:08:45


## Removing duplicates, and ordering results

How can you produce an ordered list of the first 10 surnames in the members table? 
The list must not contain duplicates.
[Link](https://pgexercises.com/questions/basic/unique.html)

In [ ]:
query = """
    select distinct surname from cd.members order by surname limit 10 
"""
execute(query)

,surname
0,Bader
1,Baker
2,Boothe
3,Butters
4,Coplin
5,Crumpet
6,Dare
7,Farrell
8,Genting
9,GUEST


## Combining results from multiple queries

You, for some reason, want a combined list of all surnames and all facility names. 
Yes, this is a contrived example :-). Produce that list!
[Link](https://pgexercises.com/questions/basic/union.html)

In [33]:
query = """
    (select surname as name from cd.members) union (select name from cd.facilities) order by name
"""
execute(query)

,name
0,Bader
1,Badminton Court
2,Baker
3,Boothe
4,Butters
5,Coplin
6,Crumpet
7,Dare
8,Farrell
9,Genting


## Simple aggregation 

You'd like to get the signup date of your last member. How can you retrieve this information?
[Link](https://pgexercises.com/questions/basic/agg.html)

In [40]:
query = """
    select max(joindate) as latest from cd.members
"""
execute(query)

,latest
0,2012-09-26 18:08:45


## More aggregation

You'd like to get the first and last name of the last member(s) who signed up - not just the date.
How can you do that?
[Link](https://pgexercises.com/questions/basic/agg2.html)

In [43]:
query = """
    select firstname, surname, joindate
    from cd.members
    where joindate= (
        select max(joindate) from cd.members
    )
"""
execute(query)

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


# Joins and Subqueries

## Retrieve the start times of members' bookings

How can you produce a list of the start times for bookings by members named 'David Farrell'?
[Link](https://pgexercises.com/questions/joins/simplejoin.html)

In [56]:
query = """
    select starttime
    from cd.bookings as b join cd.members as m on b.memid = m.memid
    where m.firstname = 'David' and m.surname = 'Farrell'
    
"""
execute(query)

,starttime
0,2012-09-18 09:00:00
1,2012-09-18 13:30:00
2,2012-09-18 17:30:00
3,2012-09-18 20:00:00
4,2012-09-19 09:30:00
5,2012-09-19 12:00:00
6,2012-09-19 15:00:00
7,2012-09-20 11:30:00
8,2012-09-20 14:00:00
9,2012-09-20 15:30:00


## Work out the start times of bookings for tennis courts

How can you produce a list of the start times for bookings for tennis courts,
for the date '2012-09-21'? 
Return a list of start time and facility name pairings, ordered by the time.
[Link](https://pgexercises.com/questions/joins/simplejoin2.html)

In [60]:
query = """
    select b.starttime, f.name
    from cd.bookings as b inner join cd.facilities as f on b.facid = f.facid
    where f.name like '%Tennis Court%' and b.starttime >= '2012-09-21' and b.starttime < '2012-09-22'
    order by b.starttime
    
"""
execute(query)

,starttime,name
0,2012-09-21 08:00:00,Tennis Court 1
1,2012-09-21 08:00:00,Tennis Court 2
2,2012-09-21 09:30:00,Tennis Court 1
3,2012-09-21 10:00:00,Tennis Court 2
4,2012-09-21 11:30:00,Tennis Court 2
5,2012-09-21 12:00:00,Tennis Court 1
6,2012-09-21 13:30:00,Tennis Court 1
7,2012-09-21 14:00:00,Tennis Court 2
8,2012-09-21 15:30:00,Tennis Court 1
9,2012-09-21 16:00:00,Tennis Court 2


## Produce a list of all members who have recommended another member

How can you output a list of all members who have recommended another member? 
Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).
[Link](https://pgexercises.com/questions/joins/self.html)

In [7]:
query = """
    select memid, firstname, surname from cd.members where memid in  (
        select distinct r.memid
        from cd.members as m inner join cd.members as r on m.recommendedby = r.memid
    )
"""

execute(query)


,memid,firstname,surname
0,1,Darren,Smith
1,2,Tracy,Smith
2,3,Tim,Rownam
3,4,Janice,Joplette
4,5,Gerald,Butters
5,6,Burton,Tracy
6,9,Ponder,Stibbons
7,11,David,Jones
8,13,Jemima,Farrell
9,15,Florence,Bader


## Produce a list of all members, along with their recommender

How can you output a list of all members, including the individual who recommended them (if any)? 
Ensure that results are ordered by (surname, firstname).
[Link](https://pgexercises.com/questions/joins/self2.html)

In [13]:
query = """
    select m.firstname as mem_fn, m.surname as mem_ln, r.firstname as rec_fn, r.surname as rec_ln
    from cd.members as m 
    left join cd.members as r 
    on m.recommendedby = r.memid
"""

execute(query)

,mem_fn,mem_ln,rec_fn,rec_ln
0,GUEST,GUEST,None,None
1,Darren,Smith,None,None
2,Tracy,Smith,None,None
3,Tim,Rownam,None,None
4,Janice,Joplette,Darren,Smith
5,Gerald,Butters,Darren,Smith
6,Burton,Tracy,None,None
7,Nancy,Dare,Janice,Joplette
8,Tim,Boothe,Tim,Rownam
9,Ponder,Stibbons,Burton,Tracy


## Produce a list of all members who have used a tennis court

How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member formatted as a single column. 
Ensure no duplicate data, and order by the member name.
[Link](https://pgexercises.com/questions/joins/threejoin.html)

In [25]:
query = """
    select distinct concat(m.firstname,' ', m.surname) as member, f.name
    from cd.members as m 
    join cd.bookings as b
    on m.memid = b.memid
    join cd.facilities as f
    on b.facid = f.facid
    where f.name like '%Tennis Court%'
    order by member
"""

execute(query)

,member,name
0,Anne Baker,Tennis Court 2
1,Anne Baker,Tennis Court 1
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 2
5,Charles Owen,Tennis Court 1
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 2
8,David Farrell,Tennis Court 1
9,David Jones,Tennis Court 1


## Produce a list of costly bookings

How can you produce a list of bookings on the day of 2012-09-14 
which will cost the member (or guest) more than $30? 
Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), 
and the guest user is always ID 0. 
Include in your output the name of the facility, the name of the member formatted as a single column,
and the cost. Order by descending cost, and do not use any subqueries.
[Link](https://pgexercises.com/questions/joins/threejoin2.html)

## Produce a list of all members, along with their recommender, using no joins.

How can you output a list of all members, including the individual who recommended them (if any), 
without using any joins? 
Ensure that there are no duplicates in the list, 
and that each firstname + surname pairing is formatted as a column and ordered.
[Link](https://pgexercises.com/questions/joins/sub.html)


## Produce a list of costly bookings, using a subquery

The Produce a list of costly bookings exercise contained some messy logic:
we had to calculate the booking cost in both the WHERE clause and the CASE statement. 
Try to simplify this calculation using subqueries. For reference, the question was:

*How can you produce a list of bookings on the day of 2012-09-14 
which will cost the member (or guest) more than $30? 
Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), 
and the guest user is always ID 0. 
Include in your output the name of the facility, 
the name of the member formatted as a single column, and the cost. Order by descending cost.*

[Link](https://pgexercises.com/questions/joins/tjsub.html)

# Modifying data


## Insert some data into a table

The club is adding a new facility - a spa. 
We need to add it into the facilities table. Use the following values:

* facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.

[Link](https://pgexercises.com/questions/updates/insert.html)

## Insert multiple rows of data into a table

In the previous exercise, you learned how to add a facility. 
Now you're going to add multiple facilities in one command. Use the following values:

* facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.
* facid: 10, Name: 'Squash Court 2', membercost: 3.5, guestcost: 17.5, initialoutlay: 5000, monthlymaintenance: 80.

[Link](https://pgexercises.com/questions/updates/insert2.html)

## Insert calculated data into a table

Let's try adding the spa to the facilities table again. 
This time, though, we want to automatically generate the value for the next facid, 
rather than specifying it as a constant. 
Use the following values for everything else:

* Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.

[Link](https://pgexercises.com/questions/updates/insert3.html)


## Update some existing data

We made a mistake when entering the data for the second tennis court. 
The initial outlay was 10000 rather than 8000: you need to alter the data to fix the error.
[Link](https://pgexercises.com/questions/updates/update.html)

## Update multiple rows and columns at the same time

We want to increase the price of the tennis courts for both members and guests. 
Update the costs to be 6 for members, and 30 for guests.
[Link](https://pgexercises.com/questions/updates/updatemultiple.html)

## Update a row based on the contents of another row

We want to alter the price of the second tennis court so that it costs 10% more than the first one. 
Try to do this without using constant values for the prices, 
so that we can reuse the statement if we want to.

[Link](https://pgexercises.com/questions/updates/updatecalculated.html)



## Delete all bookings

As part of a clearout of our database, we want to delete all bookings from the cd.bookings table.
How can we accomplish this?

[Link](https://pgexercises.com/questions/updates/delete.html)

## Delete a member from the cd.members table

We want to remove member 37, who has never made a booking, from our database.
How can we achieve that?

[Link](https://pgexercises.com/questions/updates/deletewh.html)

## Delete based on a subquery

In our previous exercises, we deleted a specific member who had never made a booking.
How can we make that more general, to delete all members who have never made a booking?
[Link](https://pgexercises.com/questions/updates/deletewh2.html)

# Aggregates


## Count the number of facilities

For our first foray into aggregates, we're going to stick to something simple.
We want to know how many facilities exist - simply produce a total count.
[Link](https://pgexercises.com/questions/aggregates/count.html)

## Count the number of expensive facilities

Produce a count of the number of facilities that have a cost to guests of 10 or more.
[Link](https://pgexercises.com/questions/aggregates/count2.html)


## Count the number of recommendations each member makes.

Produce a count of the number of recommendations each member has made. Order by member ID.
[Link](https://pgexercises.com/questions/aggregates/count3.html)

## List the total slots booked per facility

Produce a list of the total number of slots booked per facility.
For now, just produce an output table consisting of facility id and slots, sorted by facility id.
[Link](https://pgexercises.com/questions/aggregates/fachours.html)

## List the total slots booked per facility in a given month

Produce a list of the total number of slots booked per facility in the month of September 2012.
Produce an output table consisting of facility id and slots, sorted by the number of slots.
[Link](https://pgexercises.com/questions/aggregates/fachoursbymonth.html)


## List the total slots booked per facility per month

Produce a list of the total number of slots booked per facility per month in the year of 2012.
Produce an output table consisting of facility id and slots, sorted by the id and month.
[Question](https://pgexercises.com/questions/aggregates/fachoursbymonth2.html)



## Find the count of members who have made at least one booking

Find the total number of members who have made at least one booking.
[Link](https://pgexercises.com/questions/aggregates/members1.html)

## List facilities with more than 1000 slots booked

Produce a list of facilities with more than 1000 slots booked.
Produce an output table consisting of facility id and hours, sorted by facility id.
[Link](https://pgexercises.com/questions/aggregates/fachours1a.html)

## Find the total revenue of each facility

Produce a list of facilities along with their total revenue.
The output table should consist of facility name and revenue, sorted by revenue.
Remember that there's a different cost for guests and members!
[Link](https://pgexercises.com/questions/aggregates/facrev.html)

## Find facilities with a total revenue less than 1000

Produce a list of facilities with a total revenue less than 1000.
Produce an output table consisting of facility name and revenue, sorted by revenue.
Remember that there's a different cost for guests and members!
[Link](https://pgexercises.com/questions/aggregates/facrev2.html)


## Output the facility id that has the highest number of slots booked

Output the facility id that has the highest number of slots booked.
For bonus points, try a version without a LIMIT clause. This version will probably look messy!
[Link](https://pgexercises.com/questions/aggregates/fachours2.html)


## List the total slots booked per facility per month, part 2


Produce a list of the total number of slots booked per facility per month in the year of 2012. In this version, include output rows containing totals for all months per facility, and a total for all months for all facilities. The output table should consist of facility id, month and slots, sorted by the id and month. When calculating the aggregated values for all months and all facids, return null values in the month and facid columns.
[Link](https://pgexercises.com/questions/aggregates/fachoursbymonth3.html)

## List the total hours booked per named facility

Produce a list of the total number of hours booked per facility,
remembering that a slot lasts half an hour. 
The output table should consist of the facility id, name, and hours booked, sorted by facility id. 
Try formatting the hours to two decimal places.
[Link](https://pgexercises.com/questions/aggregates/fachours3.html)


## List each member's first booking after September 1st 2012

Produce a list of each member name, id, and their first booking after September 1st 2012.
Order by member ID.
[Link](https://pgexercises.com/questions/aggregates/nbooking.html)


## Produce a list of member names, with each row containing the total member count

Produce a list of member names, with each row containing the total member count. Order by join date.
[Link](https://pgexercises.com/questions/aggregates/countmembers.html)

## Produce a numbered list of members

Produce a monotonically increasing numbered list of members, ordered by their date of joining.
Remember that member IDs are not guaranteed to be sequential.
[Link](https://pgexercises.com/questions/aggregates/nummembers.html)

## Output the facility id that has the highest number of slots booked, again

Output the facility id that has the highest number of slots booked.
Ensure that in the event of a tie, all tieing results get output.
[Link](https://pgexercises.com/questions/aggregates/fachours4.html)

## Rank members by (rounded) hours used

Produce a list of members, along with the number of hours they've booked in facilities,
rounded to the nearest ten hours.
Rank them by this rounded figure, producing output of first name, surname, rounded hours, rank.
Sort by rank, surname, and first name.

[Link](https://pgexercises.com/questions/aggregates/rankmembers.html)


## Find the top three revenue generating facilities

Produce a list of the top three revenue generating facilities (including ties).
Output facility name and rank, sorted by rank and facility name.
[Link](https://pgexercises.com/questions/aggregates/facrev3.html)

## Classify facilities by value

Classify facilities into equally sized groups of high, average, and low based on their revenue.
Order by classification and facility name.
[Link](https://pgexercises.com/questions/aggregates/classify.html)

## Calculate the payback time for each facility

Based on the 3 complete months of data so far,
calculate the amount of time each facility will take to repay its cost of ownership.
Remember to take into account ongoing monthly maintenance.
Output facility name and payback time in months, order by facility name.
Don't worry about differences in month lengths, we're only looking for a rough value here!
[Link](https://pgexercises.com/questions/aggregates/payback.html)


## Calculate a rolling average of total revenue

For each day in August 2012, calculate a rolling average of total revenue over the previous 15 days.
Output should contain date and revenue columns, sorted by the date.
Remember to account for the possibility of a day having zero revenue.
This one's a bit tough, so don't be afraid to check out the hint!
[Link](https://pgexercises.com/questions/aggregates/rollingavg.html)

# Date